In [3]:
%cd ..
from agent import agent

d:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres


In [12]:
# query = "What is the Q1 2025 revenue target?"
query = "When was NeuralFlow AI founded"

### streaming: agent progress

In [9]:
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'name': 'retrieve_context', 'args': {'query': 'Q1 2025 revenue target'}, 'id': 'call_FxKbxb2w58gwoGETjZYEjCaq'}]
step: tools
content: [{'type': 'text', 'text': "Source: {'source': '..\\\\documents\\\\raw\\\\q4-2024-business-review.pdf', 'chunk_index': 144, 'source_name': 'q4-2024-business-review.pdf', 'document_chunk_index': 23, 'total_chunks_in_document': 25}\nContent: Financial Targets Q1 2025\nMetric, 1 = Q4 2024 Actual. Metric, 2 = Q1 2025 Target. Metric, 3 = Growth. Revenue, 1 = $2.8M. Revenue, 2 = $3.4M. Revenue, 3 = +21%. New Clients, 1 = 14. New Clients, 2 = 16. New Clients, 3 = +14%. MRR, 1 = $890K. MRR, 2 = $1.15M. MRR, 3 = +29%. Gross Margin, 1 = 68%. Gross Margin, 2 = 70%. Gross Margin, 3 = +2pp\n\nSource: {'source': '..\\\\documents\\\\raw\\\\meeting-notes-2025-01-08.docx', 'chunk_index': 78, 'source_name': 'meeting-notes-2025-01-08.docx', 'document_chunk_index': 19, 'total_chunks_in_document': 23}\nContent: Q1 Revenue Targets\nP

### streaming: LLM tokens

In [11]:
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="messages",
):
    # print(f"node: {metadata['langgraph_node']}")
    # print(f"content: {token.content_blocks}")
    # print("\n")
    print(token.content, end="", flush=True)

Source: {'source': '..\\documents\\raw\\q4-2024-business-review.pdf', 'chunk_index': 144, 'source_name': 'q4-2024-business-review.pdf', 'document_chunk_index': 23, 'total_chunks_in_document': 25}
Content: Financial Targets Q1 2025
Metric, 1 = Q4 2024 Actual. Metric, 2 = Q1 2025 Target. Metric, 3 = Growth. Revenue, 1 = $2.8M. Revenue, 2 = $3.4M. Revenue, 3 = +21%. New Clients, 1 = 14. New Clients, 2 = 16. New Clients, 3 = +14%. MRR, 1 = $890K. MRR, 2 = $1.15M. MRR, 3 = +29%. Gross Margin, 1 = 68%. Gross Margin, 2 = 70%. Gross Margin, 3 = +2pp

Source: {'source': '..\\documents\\raw\\meeting-notes-2025-01-08.docx', 'chunk_index': 78, 'source_name': 'meeting-notes-2025-01-08.docx', 'document_chunk_index': 19, 'total_chunks_in_document': 23}
Content: Q1 Revenue Targets
Product, 1 = Target New Customers. Product, 2 = Target MRR Growth. DocFlow AI, 1 = 12. DocFlow AI, 2 = $180K →$270K. ConversePro, 1 = 15. ConversePro, 2 = $0 →$225K. Custom Solutions, 1 = 8. Custom Solutions, 2 = $710K →$855

### streaming: custom

In [8]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": query}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: What is the Q1 2025 revenue target?
Calling tools: ['retrieve_context']
Agent: Source: {'source': '..\\documents\\raw\\q4-2024-business-review.pdf', 'chunk_index': 144, 'source_name': 'q4-2024-business-review.pdf', 'document_chunk_index': 23, 'total_chunks_in_document': 25}
Content: Financial Targets Q1 2025
Metric, 1 = Q4 2024 Actual. Metric, 2 = Q1 2025 Target. Metric, 3 = Growth. Revenue, 1 = $2.8M. Revenue, 2 = $3.4M. Revenue, 3 = +21%. New Clients, 1 = 14. New Clients, 2 = 16. New Clients, 3 = +14%. MRR, 1 = $890K. MRR, 2 = $1.15M. MRR, 3 = +29%. Gross Margin, 1 = 68%. Gross Margin, 2 = 70%. Gross Margin, 3 = +2pp

Source: {'source': '..\\documents\\raw\\meeting-notes-2025-01-08.docx', 'chunk_index': 78, 'source_name': 'meeting-notes-2025-01-08.docx', 'document_chunk_index': 19, 'total_chunks_in_document': 23}
Content: Q1 Revenue Targets
Product, 1 = Target New Customers. Product, 2 = Target MRR Growth. DocFlow AI, 1 = 12. DocFlow AI, 2 = $180K →$270K. ConversePro, 1 = 15. 

### Note:
Since, it is difficult to get the 'streaming' to work according to my expectations, I am going to take another approch where I invoke the agent and get the final response first, then using some typical python techniques, I get the final response streaming out, as follows:

In [13]:
result = agent.invoke({
    "messages": [{"role": "user", "content": query}],
})

agent_response = result['messages'][-1].content

#### option 1:

In [ ]:
import time

for token in agent_response.split():
    print(token, end=" ", flush=True)
    time.sleep(0.1)

The Q1 2025 revenue target is $3.4 million. 

#### option 2:

In [14]:
import time 

for char in agent_response:
    print(char, end='', flush=True)
    time.sleep(0.02)

NeuralFlow AI was founded in January 2023.